In [2]:
from pymongo import MongoClient, GEOSPHERE
from pprint import pprint

client = MongoClient()
db = client.geo
states = db.states
airports = db.airports
restaurants = db.restaurants
zips=db.small_zips

# Exercise 2-1

In [ ]:
code = zips.find_one({"_id":"10044"})['state']
pprint(code)

In [ ]:
area = states.find_one({"code": code})

In [ ]:
print("The state is", area['name'])

In [ ]:
cursor = airports.find(
    {"loc": { 
        "$geoWithin" : { 
            "$geometry" : area["loc"]}},
     "type": "International"},
    {"name":1, "_id" : 0}).sort("name", 1)
pprint(list(cursor))

# Exercise 2-2

In [ ]:
restaurants.create_index([("address.coord", GEOSPHERE)])

In [3]:
restaurants.index_information()

{'_id_': {'key': [('_id', 1)], 'ns': 'geo.restaurants', 'v': 2},
 'address.coord_2dsphere': {'2dsphereIndexVersion': 3,
  'key': [('address.coord', '2dsphere')],
  'ns': 'geo.restaurants',
  'v': 2},
 'address.zipcode_1': {'key': [('address.zipcode', 1)],
  'ns': 'geo.restaurants',
  'v': 2},
 'grades.grade_1': {'key': [('grades.grade', 1)],
  'ns': 'geo.restaurants',
  'v': 2},
 'name_text': {'default_language': 'english',
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'language_override': 'language',
  'ns': 'geo.restaurants',
  'textIndexVersion': 3,
  'v': 2,
  'weights': SON([('name', 1)])}}

In [4]:
area = airports.find({"type":"International"})

for airport in area:
    doc = restaurants.find(
    { "cuisine":"Korean",
     "address.coord": {
         "$near" : { 
             "$geometry" : airport["loc"],
             "$maxDistance":2000}}})
    li = list(doc)
    if(len(li)>0):
        pprint(li)

[{'_id': ObjectId('5c8626729f44c9f6767fe0af'),
  'address': {'building': '0',
              'coord': [-73.7822056, 40.6434612],
              'street': 'Jfk International Airpor',
              'zipcode': '11430'},
  'borough': 'Queens',
  'cuisine': 'Korean',
  'grades': [{'date': datetime.datetime(2014, 6, 18, 0, 0),
              'grade': 'A',
              'score': 2},
             {'date': datetime.datetime(2014, 1, 14, 0, 0),
              'grade': 'A',
              'score': 12},
             {'date': datetime.datetime(2013, 1, 4, 0, 0),
              'grade': 'A',
              'score': 11},
             {'date': datetime.datetime(2012, 4, 25, 0, 0),
              'grade': 'A',
              'score': 13}],
  'name': 'Korean Lounge',
  'restaurant_id': '40625198'}]


# Exercise 2-3

In [5]:
lax = airports.find_one({'code': 'LAX'})
dtw = airports.find_one({'code': 'DTW'})

pprint(lax)
pprint(dtw)

{'_id': ObjectId('5c8626729f44c9f6767fd637'),
 'code': 'LAX',
 'loc': {'coordinates': [-118.408084, 33.942537], 'type': 'Point'},
 'name': 'Los Angeles Intl',
 'type': 'International'}
{'_id': ObjectId('5c8626729f44c9f6767fd564'),
 'code': 'DTW',
 'loc': {'coordinates': [-83.348836, 42.212052], 'type': 'Point'},
 'name': 'Detroit Metropolitan - Wayne County',
 'type': 'International'}


In [6]:
geojson = {
    'type': 'LineString',
    'coordinates': [lax['loc']['coordinates'],
                    dtw['loc']['coordinates']]
}
result = states.find({
    'loc': {
        '$geoIntersects': {
            '$geometry': geojson
        }
    } 
}, {'_id': 0, 'name': 1}).sort([('name', 1)])

pprint(list(result))

[{'name': 'Arizona'},
 {'name': 'California'},
 {'name': 'Colorado'},
 {'name': 'Illinois'},
 {'name': 'Iowa'},
 {'name': 'Kansas'},
 {'name': 'Michigan'},
 {'name': 'Nebraska'},
 {'name': 'Nevada'},
 {'name': 'Utah'}]


In [7]:
client.close()